von Likith https://www.kaggle.com/code/likith012/ecgnet-ptb-xl


In [8]:
import numpy as np
import pandas as pd
import wfdb
import ast
from scipy.signal import spectrogram
import math
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import keras

In [2]:
### Preprocessing
#   Using the super classes, multi label classification, excluding samples with no labels and considering atleast one label

path = '../'
Y = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')


# rdsamp gibt Tupel (signal, meta) zurück
# data wird (N, L, C) NumPy-Array (Anzahl Records, Samples pro Record, Kanäle)
data = np.array([wfdb.rdsamp(path + f)[0] for f in Y.filename_lr])

# scp_codes als String gespeichert (z. B. "{'NORM': 100}").
# ast.literal_eval wandelt solche Strings in echte Python-Dictionaries um
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# agg_df ordnet die scp_codes zu übergeordneten diagnostischen Klassen (diagnostic_class) zu.
# Mit agg_df[agg_df.diagnostic == 1] bleiben nur diagnostisch relevante Codes für Klassifikation
agg_df = pd.read_csv(path + 'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]


# Für jeden Record nimmt agg die Keys aus scp_codes und sammelt die zugehörigen diagnostic_class-Namen
# list(set(...)) entfernt Duplikate → Ergebnis: Liste mit 0, 1 oder mehreren Superklassen pro Record
def agg(y_dic):
    temp = []

    for key in y_dic.keys():
        if key in agg_df.index:  # gibt es den Code in der Mapping-Tabelle?
            c = agg_df.loc[key].diagnostic_class  # holt die Oberklasse
            if str(c) != 'nan':  # prüft, ob es nicht leer/NaN ist
                temp.append(c)  # fügt die Oberklasse zu temp hinzu
    return list(set(temp))  # 5) gibt eindeutige Liste zurück


# diagnostic_superclass: z. B. ['NORM'] oder ['MI'] oder ['NORM','STTC'].
# superdiagnostic_len: Anzahl Superklassen pro Record (0,1,>1)
Y['diagnostic_superclass'] = Y.scp_codes.apply(agg)
Y['superdiagnostic_len'] = Y['diagnostic_superclass'].apply(lambda x: len(x))


# nur diese drei Klassen behalten und nur Records mit GENAU 1 Superklasse (aus Multi-Label wird Single-Label)
keep_classes = {'NORM', 'STTC', 'CD'}
mask = Y['diagnostic_superclass'].apply(
    lambda xs: len(xs) == 1 and set(xs).issubset(keep_classes)
)

# gefilterte Signale + Metadaten
X_data = data[mask]        # (N, 1000, 12)
Y_data = Y[mask].copy()


# One-Hot-Encoding der Labels
mlb = MultiLabelBinarizer(classes=['CD','NORM','STTC'])  # feste Reihenfolge
y = mlb.fit_transform(Y_data['diagnostic_superclass'])
print("Klassen:", list(mlb.classes_))  # ['CD','NORM','STTC']


########

## Stratify split

# PTB-XL stellt strat_fold bereit (Folds 1–10).

# strat_fold < 9 → Trainingsdaten (typisch Folds 1–8),

# strat_fold == 9 → Validation,

# strat_fold == 10 → Test.


X_train = X_data[Y_data.strat_fold < 9]
y_train = y[Y_data.strat_fold < 9]

X_val = X_data[Y_data.strat_fold == 9]
y_val = y[Y_data.strat_fold == 9]

X_test = X_data[Y_data.strat_fold == 10]
y_test = y[Y_data.strat_fold == 10]




# Löscht temporäre Variablen, um RAM freizugeben
del X_data, Y_data, y

#########

# Standardizing

def apply_scaler(X, scaler):
    X_tmp = []
    for x in X:
        x_shape = x.shape
        X_tmp.append(scaler.transform(x.flatten()[:, np.newaxis]).reshape(x_shape))
    X_tmp = np.array(X_tmp)
    return X_tmp


scaler = StandardScaler()

scaler.fit(np.vstack(X_train).flatten()[:, np.newaxis].astype(float))

X_train = apply_scaler(X_train, scaler)
X_val = apply_scaler(X_val, scaler)
X_test = apply_scaler(X_test, scaler)


Klassen: ['CD', 'NORM', 'STTC']


In [18]:
import joblib
# speichert die vorverarbeiteten Daten, den Scaler und die Klassenliste
np.savez("../ptbxl_preprocessed.npz",
         X_train=X_train, y_train=y_train,
         X_val=X_val, y_val=y_val,
         X_test=X_test, y_test=y_test)
joblib.dump(scaler, "../scaler.pkl")
joblib.dump(list(mlb.classes_), "../classes.pkl")


['classes.pkl']

In [33]:
import joblib
# lädt die vorverarbeiteten Daten, den Scaler und die Klassenliste
data = np.load("../artifacts/ptbxl_preprocessed.npz")
X_train, y_train = data["X_train"], data["y_train"]
X_val, y_val     = data["X_val"],   data["y_val"]
X_test, y_test   = data["X_test"],  data["y_test"]

scaler = joblib.load("../artifacts/scaler.pkl")
classes = joblib.load("../artifacts/classes.pkl")


In [15]:
print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("X_test:", X_test.shape)

X_train: (10499, 1000)
X_val: (1340, 1000)
X_test: (1338, 1000)


In [24]:
'''
X_train = X_train[..., None]
X_val   = X_val[..., None]
X_test  = X_test[..., None]
'''

In [25]:
# Anzahl Samples pro Klasse anzeigen
count_class = list(classes)

def show_counts(y, name):
    cnt = y.sum(axis=0).astype(int)
    print(f"\n{name} counts:")
    for c, n in zip(count_class, cnt):
        print(f"{c:>5}: {n}")
    print("total:", int(cnt.sum()))

show_counts(y_train, "Train")
show_counts(y_val,   "Val")
show_counts(y_test,  "Test")


Train counts:
   CD: 1353
 NORM: 7243
 STTC: 1903
total: 10499

Val counts:
   CD: 171
 NORM: 914
 STTC: 255
total: 1340

Test counts:
   CD: 184
 NORM: 912
 STTC: 242
total: 1338


In [6]:
# Nur den ersten Kanal (erste EKG-Ableitung) aus den Daten nehmen.
# weil Wearable auch nur 1 Kanal hat
'''
X_train = X_train[:, :, 0]
X_val = X_val[:, :, 0]
X_test = X_test[:, :, 0]
'''



'\nX_train = X_train[:, :, 0]\nX_val = X_val[:, :, 0]\nX_test = X_test[:, :, 0]\n'

In [10]:

class DataGen(keras.utils.Sequence):
        # X = Eingabedaten (z. B. EKG-Signale)

        # y = Labels (One-Hot oder Integer)

        # batch_size = wie viele Samples pro Schritt an das Modell gehen

        # sampling = Abtastrate des Signals (z. B. 100 Hz)

        # window_len + overlap_len = Parameter für das Spektrogramm (siehe unten)
    def __init__(self, X, y, batch_size=32, window_len=40, overlap_len=10, sampling=100, **kwargs):
        super().__init__(**kwargs)
        self.batch_size = batch_size
        self.X = X
        self.y = y
        self.sampling = sampling
        self.window_len = window_len
        self.overlap_len = overlap_len

    # Gibt an, wie viele Batches pro Epoche existieren
    def __len__(self):
        return math.ceil(len(self.X) / self.batch_size)

    # Schneidet ein Stück (Batch) aus den Daten raus. batch_x = Signale, batch_y = zugehörige Labels
    def __getitem__(self, idx):
        batch_x = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]



        spectrogram_data = []
        for item in batch_x:

            #_, _, Sxx = spectrogram(item, fs=100, nperseg=40, noverlap=10)
            #_, _, Sxx = spectrogram(sig.astype(np.float32), fs=100, nperseg=40, noverlap=10)
            #_,_,Sxx = spectrogram(item, fs = self.sampling,  nperseg = self.window_len, noverlap= self.overlap_len)
            #Sxx = Sxx[:13].transpose()
            #Sxx = np.log(Sxx + 1e-8)  # log, avoid zero
            #Sxx = np.log(Sxx[:13] + 1e-8).T

            # hinzugefügt wegen Fehlermeldung
            sig  = np.asarray(item).squeeze().astype(np.float32)  # (1000,1) -> (1000,)
            nper = min(self.window_len, sig.shape[0])
            nover = min(self.overlap_len, max(0, nper - 1))

            f, t, Sxx = spectrogram(sig, fs=self.sampling, nperseg=nper, noverlap=nover)
            Sxx = Sxx[:13].transpose()
            Sxx = np.where(Sxx > 0, np.log(Sxx), 0.0)


            Sxx = abs(Sxx)
            mask = Sxx > 0
            Sxx[mask] = np.log(Sxx[mask])
            spectrogram_data.append(Sxx)

        # X_1: Rohsignal umformen auf (batch_size, 1000, 1) → für ein 1D-CNN.
        # X_2: Spektrogramme als NumPy-Array → für ein 2D-CNN.
        # final_X = (X_1, X_2) → ein Tupel von zwei Inputs
        X_1 = batch_x.reshape(-1, 1000, 1)

        X_2 = np.array(spectrogram_data)


        final_X = (X_1,X_2)


        return final_X ,batch_y




window_len = 40
overlap_len = 10

train_gen = DataGen(X_train, y_train, window_len=window_len, overlap_len=overlap_len)
test_gen = DataGen(X_val, y_val, window_len=window_len, overlap_len=overlap_len)


In [11]:
import keras
from keras import layers, ops


@keras.saving.register_keras_serializable(name="attention")
class attention(layers.Layer):

    def __init__(self, return_sequences=True, **kwargs):
        super().__init__(**kwargs)
        self.return_sequences = return_sequences
        #super(attention, self).__init__()

    def build(self, input_shape):
        '''
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(attention, self).build(input_shape
        '''
        time = int(input_shape[1])
        feat = int(input_shape[2])
        self.W = self.add_weight(name="att_weight", shape=(feat, 1), initializer="glorot_uniform")
        self.b = self.add_weight(name="att_bias", shape=(time, 1), initializer="zeros")
        super().build(input_shape)

    def call(self, x):
        e = ops.tanh(ops.dot(x, self.W) + self.b)
        a = ops.softmax(e, axis=1)
        output = x * a

        if self.return_sequences:
            return output

        return ops.sum(output, axis=1)

    def get_config(self):
        cfg = super().get_config()
        cfg.update({"return_sequences": self.return_sequences})
        return cfg


from keras.layers import Conv1D, Input, Attention, LSTM, Activation, Dense, Average


inputs_1 = Input(shape=(1000, 1), batch_size=None)

x = Conv1D(32, 2, padding='same')(inputs_1)
x = Activation('relu')(x)

x = Conv1D(64, 2, padding='same')(x)
x = Activation('relu')(x)

x = attention(return_sequences=True)(x)

x = LSTM(64, return_sequences=True)(x)
x = LSTM(64)(x)

output_1 = Dense(256, activation='relu')(x)


inputs_2 = Input(shape=(33, 13), batch_size=None)

u = Conv1D(6, 2, padding='same')(inputs_2)
u = Activation('relu')(u)

u = Conv1D(16, 4, padding='same')(u)
u = Activation('relu')(u)

u = LSTM(256, name='lstm_spec')(u)
output_2 = Dense(256, activation='relu')(u)


avg = Average()([output_1, output_2])


#outputs = Dense(5, activation='softmax')(avg)
outputs = Dense(3, activation='softmax')(avg) # 3 Klassen: CD, NORM, STTC


model = keras.models.Model(inputs=[inputs_1, inputs_2], outputs=outputs)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy', keras.metrics.AUC()])

# Zeigt dir alle Layer, Shapes und Anzahl Parameter.
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1000, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 1000, 32)  │         96 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1000, 32)  │          0 │ conv1d[0][0]      │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 33, 13)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 1000, 64)  │      4,160 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 33, 6)     │        162 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 1000, 64)  │          0 │ conv1d_1[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 33, 6)     │          0 │ conv1d_2[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 1000, 64)  │      1,064 │ activation_1[0][… │
│ (attention)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 33, 16)    │        400 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 1000, 64)  │     33,024 │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 33, 16)    │          0 │ conv1d_3[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     33,024 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_spec (LSTM)    │ (None, 256)       │    279,552 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     16,640 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ lstm_spec[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average (Average)   │ (None, 256)       │          0 │ dense[0][0],      │
│                     │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 3)         │        771 │ average[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 434,685 (1.66 MB)

 Trainable params: 434,685 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
bx, by = train_gen[0]
print(bx[0].shape)   # Rohsignale
print(bx[1].shape)   # Spektrogramm
print(by.shape)      # Labels


(32, 1000, 1)
(32, 33, 13)
(32, 3)


In [12]:
cbs = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=2, factor=0.5, verbose=1),

    keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=4, restore_best_weights=True, verbose=1),

    keras.callbacks.ModelCheckpoint('../best.keras', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
]

# Klassengewichte: CD stärker gewichten, STTC leicht stärken weil deren Recall noch zu schwach
class_weight = {0: 2.0, 1: 1.0, 2: 1.2}

In [14]:
model.fit(train_gen, validation_data=test_gen, epochs=50, callbacks=cbs, class_weight=class_weight)

Epoch 1/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - accuracy: 0.6902 - auc_1: 0.7803 - loss: 1.0934
Epoch 1: val_loss improved from None to 0.81738, saving model to best.keras
329/329 ━━━━━━━━━━━━━━━━━━━━ 252s 749ms/step - accuracy: 0.7017 - auc_1: 0.7912 - loss: 1.0713 - val_accuracy: 0.7060 - val_auc_1: 0.8126 - val_loss: 0.8174 - learning_rate: 0.0010
Epoch 2/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step - accuracy: 0.7188 - auc_1: 0.8183 - loss: 1.0116
Epoch 2: val_loss improved from 0.81738 to 0.80989, saving model to best.keras
329/329 ━━━━━━━━━━━━━━━━━━━━ 165s 501ms/step - accuracy: 0.7177 - auc_1: 0.8173 - loss: 1.0128 - val_accuracy: 0.7127 - val_auc_1: 0.8237 - val_loss: 0.8099 - learning_rate: 0.0010
Epoch 3/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step - accuracy: 0.7218 - auc_1: 0.8297 - loss: 0.9916
Epoch 3: val_loss improved from 0.80989 to 0.79249, saving model to best.keras
329/329 ━━━━━━━━━━━━━━━━━━━━ 155s 472ms/step - accuracy: 0.7212 - auc_1: 0.8326 - loss: 0.98

AB HIER SELBST ERGÄNZT: TESTS ETC

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score

# Klassennamen
class_names = list(classes)

# Labels
y_true_oh = test_gen.y
y_true    = y_true_oh.argmax(axis=1)

# Vorhersagen
probs = model.predict(test_gen, verbose=0)
y_pred = probs.argmax(axis=1)

# Precision, Recall, F1-Score für jede Klasse
print(classification_report(y_true, y_pred, target_names=class_names))

# Konfusionsmatrix
print(confusion_matrix(y_true, y_pred))

# AUROC
auc_macro = roc_auc_score(y_true_oh, probs, multi_class='ovr', average='macro')
print("Macro AUROC:", auc_macro)

# AUROC pro Klasse
auc_per_class = roc_auc_score(y_true_oh, probs, multi_class='ovr', average=None)
for name, auc in zip(class_names, auc_per_class):
    print(f"AUC[{name}]: {auc:.3f}")

# Macro F1-Score
f1_macro = f1_score(y_true, y_pred, average='macro')
print("Macro F1:", f1_macro)


              precision    recall  f1-score   support

          CD       0.13      1.00      0.23       171
        NORM       0.00      0.00      0.00       914
        STTC       0.00      0.00      0.00       255

    accuracy                           0.13      1340
   macro avg       0.04      0.33      0.08      1340
weighted avg       0.02      0.13      0.03      1340

[[171   0   0]
 [914   0   0]
 [255   0   0]]
Macro AUROC: 0.40413834369464335
AUC[CD]: 0.372
AUC[NORM]: 0.398
AUC[STTC]: 0.442
Macro F1: 0.07544672402382528


C:\Users\milal\IdeaProjects\Projektarbeit_KI_EKG\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\milal\IdeaProjects\Projektarbeit_KI_EKG\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\milal\IdeaProjects\Projektarbeit_KI_EKG\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [16]:
# Speichert das beste Modell (bereits durch ModelCheckpoint geschehen, hier nur zur Sicherheit)
model.save("best.keras")


In [16]:
import numpy as np
from sklearn.metrics import precision_recall_curve

def best_thr_Fbeta(y_true_bin, scores, beta=2.0):
    P, R, T = precision_recall_curve(y_true_bin, scores)
    fb = (1+beta**2) * (P*R) / (beta**2*P + R + 1e-12)
    i = np.nanargmax(fb[:-1])      # letztes P/R gehört zu T-undefiniert
    return float(T[i])

# Val-Set
probs_val = model.predict(test_gen, verbose=0)    # (N,3)
y_val_int = np.concatenate([y for _, y in test_gen]).argmax(1)


thr_cd   = best_thr_Fbeta((y_val_int==0).astype(int), probs_val[:,0], beta=2.0)
thr_sttc = best_thr_Fbeta((y_val_int==2).astype(int), probs_val[:,2], beta=1.5)
thr_norm = 0.50  # meist bei 0.5 lassen
print(thr_cd, thr_sttc)


0.34118518233299255 0.3229576349258423


In [17]:
probs = model.predict(test_gen, verbose=0)       # (N_gen, 3)
y_true = np.concatenate([y for _, y in test_gen]).argmax(1)

thr_cd, thr_sttc = 0.19, 0.22
y_pred = []
for p in probs:
    if p[0] >= thr_cd:   y_pred.append(0)     # CD
    elif p[2] >= thr_sttc: y_pred.append(2)  # STTC
    else:                y_pred.append(int(np.argmax(p)))
y_pred = np.array(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=['CD','NORM','STTC']))


              precision    recall  f1-score   support

          CD       0.13      1.00      0.23       171
        NORM       0.00      0.00      0.00       914
        STTC       0.00      0.00      0.00       255

    accuracy                           0.13      1340
   macro avg       0.04      0.33      0.08      1340
weighted avg       0.02      0.13      0.03      1340



C:\Users\milal\IdeaProjects\Projektarbeit_KI_EKG\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\milal\IdeaProjects\Projektarbeit_KI_EKG\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\milal\IdeaProjects\Projektarbeit_KI_EKG\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [19]:
# Modell laden

model = keras.saving.load_model(
    "../artifacts/best.keras",
    custom_objects={"attention": attention},
    safe_mode=False,
    compile=False
)
model.summary()




Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1000, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 1000, 32)  │         96 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 1000, 32)  │          0 │ conv1d_4[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 33, 13)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 1000, 64)  │      4,160 │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 33, 6)     │        162 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 1000, 64)  │          0 │ conv1d_5[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 33, 6)     │          0 │ conv1d_6[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, 1000, 64)  │      1,064 │ activation_5[0][… │
│ (attention)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 33, 16)    │        400 │ activation_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 1000, 64)  │     33,024 │ attention_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 33, 16)    │          0 │ conv1d_7[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 64)        │     33,024 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_spec (LSTM)    │ (None, 256)       │    279,552 │ activation_7[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256)       │     16,640 │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     65,792 │ lstm_spec[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_1 (Average) │ (None, 256)       │          0 │ dense_3[0][0],    │
│                     │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 3)         │        771 │ average_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 434,685 (1.66 MB)

 Trainable params: 434,685 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# HDF5-Datei öffnen und Daten inspizieren
import h5py


dateipfad = '../cachet-cadb_short_format_without_context.hdf5'

try:

    with h5py.File(dateipfad, 'r') as f:

        print("Gruppen in der Datei:", list(f.keys()))

        for name in f.keys():
            gruppe = f[name]
            print(f"Inhalt von {gruppe}: {gruppe[:]}")

except FileNotFoundError:
    print(f"Fehler: Die Datei '{dateipfad}' wurde nicht gefunden.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")


Gruppen in der Datei: ['labels', 'signal']
Inhalt von <HDF5 dataset "labels": shape (16404480,), type "<f8">: [3. 3. 3. ... 3. 3. 3.]
Inhalt von <HDF5 dataset "signal": shape (16404480,), type "<f8">: [-0.00135984  0.00279032  0.00586285 ... -0.0378265  -0.03591219
 -0.03357666]


In [27]:
import h5py

file_name = '../cachet-cadb_short_format_without_context.hdf5'
f1 = h5py.File(file_name,'r')

print("Top-level keys:", list(f1.keys()))

'''
first_key = list(f1.keys())[0]
dataset = f1[first_key]

print("Typ:", type(dataset))
print("Shape:", dataset.shape)
print("Dtype:", dataset.dtype)

# Daten ins NumPy-Array laden
data = dataset[:]
print("Array:", data)
'''

# Signal-Daten anschauen
signal = f1['signal'][:]   # als NumPy-Array laden
print(signal.shape, signal.dtype)
print(signal[:10])         # erste 10 Werte

# Labels anschauen
labels = f1['labels'][:]
print(labels.shape, labels.dtype)
print(labels[:20])        # erste 20 Werte


Top-level keys: ['labels', 'signal']
(16404480,) float64
[-0.00135984  0.00279032  0.00586285  0.00804016  0.00948632  0.01034809
  0.01075599  0.01082523  0.01065672  0.010338  ]
(16404480,) float64
[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


In [42]:
# Testladen der Klassen und des Scalers
import joblib

try:
    with open('../artifacts/classes.pkl', 'rb') as file:
        geladene_daten = joblib.load(file)


    print(geladene_daten)

except FileNotFoundError:
    print("Fehler: Die Datei 'classes.pkl' wurde nicht gefunden.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")


try:
    with open('../artifacts/scaler.pkl', 'rb') as file:
        geladene_daten = joblib.load(file)


    print(geladene_daten)

except FileNotFoundError:
    print("Fehler: Die Datei 'scaler.pkl' wurde nicht gefunden.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")

['CD', 'NORM', 'STTC']
StandardScaler()


In [59]:
import h5py

import keras
from ecg_project.ecg_preprocess import standardizing
from ecg_project.model import predict_ecg

fs = 100
samples = 10 * fs


model = keras.models.load_model("../artifacts/best.keras", compile=False, custom_objects={"attention": Attention})

classes = joblib.load("../artifacts/classes.pkl")
print("Klassen:", classes)


with h5py.File("../cachet-cadb_short_format_without_context.hdf5", "r") as f:
    sig = f["signal"][:]
    labels = f["labels"][:]

seg_idx = 0
ecg = sig[seg_idx*samples:(seg_idx+1)*samples]

CACHET_LABELS = {
    1: "AF",
    2: "NSR",
    3: "Noise",
    4: "Others"
}

lbl_num = int(labels[seg_idx])
lbl_name = CACHET_LABELS[lbl_num]



ecg2d = standardizing(ecg)



# Preprocess → (1,1000,1)
#X_in = make_model_inputs(ecg2d)

'''
# Prediction
preds = model.predict(X_in)

print(labels)
print("Vorhersage für Segment", seg_idx, ":", preds, "vorhergesagte Diagnose:", classes[np.argmax(preds)], "mit Wahrscheinlichkeit von", float(np.max(preds)), "Label:", labels[seg_idx])
'''
predictions = predict_ecg(ecg2d)
print(predictions)
print("eigentliche Klasse:", lbl_name)


Klassen: ['CD', 'NORM', 'STTC']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
('NORM', 50.04)
eigentliche Klasse: Noise


In [70]:
# Test mit einer Apple Watch Datei
import wfdb
import numpy as np
from ecg_project.model import predict_ecg
from ecg_project.ecg_preprocess import standardizing, resampling

rec_base = "../electrocardiogram-capable-smartwatches-assessing-their-clinical-accuracy-and-application-1.0.0/applewatch_serie8/st-segment/st-p1/st-p1_4"  # ohne .hea/.dat
rec = wfdb.rdrecord(rec_base)

#print(rec.__dict__.keys())


sig = rec.p_signal[:]
# print(sig)
fs  = int(rec.fs)
print(sig.shape, fs)
fs_new = 100
resample = resampling(sig, fs, fs_out=fs_new)

print(resample.shape, fs_new)

ecg2d = standardizing(resample)
print(ecg2d.shape)

predictions = predict_ecg(ecg2d)

print(predictions)




dict_keys(['record_name', 'n_sig', 'fs', 'counter_freq', 'base_counter', 'sig_len', 'base_time', 'base_date', 'comments', 'sig_name', 'p_signal', 'd_signal', 'e_p_signal', 'e_d_signal', 'file_name', 'fmt', 'samps_per_frame', 'skew', 'byte_offset', 'adc_gain', 'baseline', 'units', 'adc_res', 'adc_zero', 'init_value', 'checksum', 'block_size'])
(15360, 1) 512
(1000, 1, 1) 100
(1000, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
('STTC', 38.32)


In [73]:
# Test mit einer Apple Watch ZIP-Datei
from ecg_project.ecg_preprocess import load_ecg_wfdb_zip, resampling
from ecg_project.model import predict_ecg

with open("C:\\Users\\milal\\Downloads\\Projektarbeit WS2526\\electrocardiogram-capable-smartwatches-assessing-their-clinical-accuracy-and-application-1.0.0\\applewatch_serie8\\st-segment\\st-p5\\st-p5_0.zip", "rb") as f:
    content = f.read()

sig, fs = load_ecg_wfdb_zip(content)
print("Signal-Shape:", sig.shape)
print("Samplingrate:", fs)

data = resampling(sig, fs)
print(data.shape)
pred = predict_ecg(data)
print(pred)


Signal-Shape: (15360,)
Samplingrate: 512
(1000, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
('NORM', 53.7)
